# Support Vector Machines
Classifing student success data by means of the [Support Vector Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) from the sklearn module.

## Import Data
Import the data into a pandas dataframe. Get dummy variables for each categorical predictor in the data set and return the design matirx. Create a normalized and standardized design matrix as well to compare model preformance. Convert response variable to three classes *0 , 1,* and *2*.

In [1]:
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectPercentile
from sklearn.utils.extmath import cartesian
from sklearn import preprocessing
from sklearn import metrics, svm

df = pd.read_csv('student-por2.csv')
df = pd.get_dummies(df)#, drop_first=True)

def response_conv(arr):
    new = []
    for i in arr:
        if (i > 0 and i < 10):           # condition where student failed
            new.append(0)                 
                                          
        elif (i >= 10):                  # condition where student passed
            new.append(1)                 
    
        else:                            # condition where student received an incomplete
            new.append(2)
    return(new)                          # 1-dimensional response varibale returned

X = df.drop('G3',1)
y = response_conv(list(df.G3))

select = SelectPercentile()
newX = select.fit_transform(X,y)         # design matrix with most influential predictors only

X_scale = preprocessing.scale(newX)
X_norm = preprocessing.normalize(newX)

random.seed(42)
X1_train, X1_test, y1_train, y1_test = train_test_split(newX, y, test_size=0.33, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_scale, y, test_size=0.33, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_norm, y, test_size=0.33, random_state=42)

C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


## Niave Accuracy
Before we start training and selecting parametrs for our model, we must find the distribution of the classes amongst the response variable. Depnding on which class is the dominate class, our model should preform better than just guessing the dominate class for each observation. For example, if the dominate class is 1 and 1's comprise of 83% of the response data, then our model should have higher than 83% accuracy. 

In [2]:
zero = 0
one = 0
two = 0

for i in y1_test:
    if i == 0:
        zero += 1
    elif i == 1:
        one += 1
    else:
        two += 1

num1 = round((zero/len(y1_test))*100,2)
num2 = round((one/len(y1_test))*100,2)
num3 = round((two/len(y1_test))*100,2)
print("The testing response vector has the following distribution: \nzeros: %r zeros comprising of %r percent of the response data. \nones: %r ones comprising of %r percent of the response data. \ntwos: %r twos comprising of %r percent of the response data." % (zero,num1,one,num2,two,num3))
print("\n")

The testing response vector has the following distribution: 
zeros: 23 zeros comprising of 10.7 percent of the response data. 
ones: 187 ones comprising of 86.98 percent of the response data. 
twos: 5 twos comprising of 2.33 percent of the response data.




## Optimal Penalty Parameter, Kernel, and Gamma
The fucntion *opt* finds the optimal parameters for *C*, *Gamma*, and *Kernel* to be used in the model. Optimal is decided based on the parameters used in the model that returns the smallest mean squared error.

In [3]:
start_time = time.time()
combos = cartesian([['linear','rbf'],[0.1,1,10,100,1000],[0.1,0.01,0.001,0.0001]])
def opt(X,y):
    acc = []

    for k,c,g in combos:
        svc = svm.SVC(C=float(c),kernel=str(k),gamma=float(g),probability=True,decision_function_shape='ovo') #,probability=True
        scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
        acc.append(scores.mean())
    
    #MSE = [1 - x for x in cv_scores]
    opt_ = combos[acc.index(max(acc))]
    return(opt_)

k1,c1,g1 = opt(X1_train,y1_train)
k2,c2,g2 = opt(X2_train,y2_train)
k3,c3,g3 = opt(X3_train,y3_train)

print ("The optimal kernel, penalty parameter and gamma is %s, %r and %r respectively for Non-standardized design matrix." % (k1,float(c1),float(g1)))
print ("The optimal kernel, penalty parameter and gamma is %s, %r and %r respectively for standardized design matrix." % (k2,float(c2),float(g2)))
print ("The optimal kernel, penalty parameter and gamma is %s, %r and %r respectively for normalized design matrix." % (k3,float(c3),float(g3)))
print("Run time: %r minutes" % (round((int(time.time() - start_time)/60),2)))

The optimal kernel, penalty parameter and gamma is rbf, 1000.0 and 0.0001 respectively for Non-standardized design matrix.
The optimal kernel, penalty parameter and gamma is rbf, 10.0 and 0.01 respectively for standardized design matrix.
The optimal kernel, penalty parameter and gamma is linear, 1000.0 and 0.1 respectively for normalized design matrix.
Run time: 2.13 minutes


## Fit and Predict
After tuning model parameters to be optimal we fit each design matrix to its optimal model. Predictions are made and returned in a data frame for comparison.

In [4]:
SVM1 = svm.SVC(C=float(c1),kernel=str(k1),gamma=float(g1),decision_function_shape='ovo').fit(X1_train,y1_train)
SVM2 = svm.SVC(C=float(c2),kernel=str(k2),gamma=float(g2),decision_function_shape='ovo').fit(X2_train,y2_train)
SVM3 = svm.SVC(C=float(c3),kernel=str(k3),gamma=float(g3),decision_function_shape='ovo').fit(X3_train,y3_train)


svm1_pred = SVM1.predict(X1_test)
svm2_pred = SVM2.predict(X2_test)
svm3_pred = SVM3.predict(X3_test)

pred = pd.DataFrame(list(zip(y1_test, svm1_pred,svm2_pred,svm3_pred)), columns=['y_act','y_svm','y_svm_stand','y_svm_norm'])
pred.index.name = 'Obs'
pred

,y_act,y_svm,y_svm_stand,y_svm_norm
Obs,,,,
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,1,1,1,1
6,1,1,1,1
7,0,1,1,1
8,1,1,1,1


## Results
Accuracy, confusion matrix, and classification reports are returned for each design matirx. All of the models seem to capture the means for predicting really well. Thus we can take the non-standardized design matrix for our final model.

In [5]:
cm_svm1 = pd.DataFrame(metrics.confusion_matrix(y1_test, svm1_pred), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_svm2 = pd.DataFrame(metrics.confusion_matrix(y2_test, svm2_pred), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_svm3 = pd.DataFrame(metrics.confusion_matrix(y3_test, svm3_pred), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])

print ("The accuracy of the Non-standardized SVM model is: ", SVM1.score(X1_test,y1_test))
print ("\n")
print ("The accuracy of the standardized SVM model is: ", SVM2.score(X2_test,y2_test))
print ("\n")
print ("The accuracy of the normalized SVM model is: ", SVM3.score(X3_test,y3_test))
print ("\n")

print("Non-standarized SVM Confusion Matrix: \n", cm_svm1)
print ("\n")
print("Standarized SVM Confusion Matrix: \n", cm_svm2)
print ("\n")
print("Normalized SVM Confusion Matrix: \n", cm_svm3)
print ("\n")

print("Classification report for Non-standardized design matrix:\n", metrics.classification_report(y1_test,svm1_pred))
print("\n")
print("Classification report for standardized design matrix:\n", metrics.classification_report(y2_test,svm2_pred))
print("\n")
print("Classification report for Normalized design matrix:\n", metrics.classification_report(y3_test,svm3_pred))

The accuracy of the Non-standardized SVM model is:  0.920930232558


The accuracy of the standardized SVM model is:  0.916279069767


The accuracy of the normalized SVM model is:  0.916279069767


Non-standarized SVM Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)       14        9       0
Pass(1)        6      181       0
Inc(2)         1        1       3


Standarized SVM Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)       14        9       0
Pass(1)        7      180       0
Inc(2)         1        1       3


Normalized SVM Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)       13       10       0
Pass(1)        6      181       0
Inc(2)         1        1       3


Classification report for Non-standardized design matrix:
              precision    recall  f1-score   support

          0       0.67      0.61      0.64        23
          1       0.95      0.97      0.96       187
          2       1.00      0.60      0.75         5

avg /